In [ ]:
from pathlib import Path

import random

from collections import Counter
import subprocess as sp
import multiprocessing as mp

from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

import uuid

In [ ]:
home = Path(os.environ['HOME']) 
p = home / 'pdfs'
pdfs = sorted([p / x for x in os.listdir(p)], key=os.path.getsize)

pkl_path = './pdfs_txt.pkl'

nlp = English()
tokenizer = Tokenizer(nlp.vocab)

In [ ]:
def extract_strings(pdf):
    cmd = ['pdfgrep', '-i', '[a-z0-9]', str(pdf)]
    out = sp.run(cmd, stdout=sp.PIPE, stderr=sp.PIPE)
    
    is_garbage = lambda t: any([
        len(t) < 3, t.is_punct, t.is_stop, t.is_space, t.like_num
    ])

    tokens = tokenizer(out.stdout.decode('utf8'))
    tokens = [re.sub(r'[^a-z]', '', t.text.lower()) for t in tokens if not is_garbage(t)]
    tokens = [t for t in tokens if len(t) > 0]
    
    return pdf, Counter(tokens)

In [ ]:
with mp.Pool(processes=8) as pool:
    out = pool.map(extract_strings, pdfs)
    
pickle.dump(out, open(pkl_path, 'wb'))